#### Выгрузка данных 
Определение кодировки, выгружаемых файлов  
Выгрузка и объдинение файлов в формате csv в один общий дата фрейм  

In [1]:
import pandas as pd
import glob
import os
from datetime import datetime, timedelta
import charset_normalizer
from datetime import time


path = (r'C:\Users\Home\Desktop\Data\Точки_продаж')

file_encoding = r'C:\Users\Home\Desktop\Data\Точки_продаж\time_live1 01-22.04.csv'
with open(file_encoding, 'rb') as difine_code:
    result = charset_normalizer.detect(difine_code.read(10000))
print(f'Наиболее вероятная кодировка: {result}')

def concat_all_csv_files(path):
    os.chdir(path)
    
    # Получаем список всех CSV файлов в папке
    csv_files = glob.glob('*.csv')
    
    # Сортируем файлы по дате изменения (от новых к старым)
    csv_files.sort(key=os.path.getmtime, reverse=True)
    
    combined = pd.DataFrame()
    
    try:
        for csv_file in csv_files:
            # Читаем CSV файл
            data = pd.read_csv(csv_file, skiprows=6, encoding='windows-1251', sep=';', parse_dates=True, on_bad_lines='warn',low_memory=False)  
            
            # Добавляем колонку с именем файла
            data['source_file'] = csv_file
            
            # Объединяем с общим DataFrame
            combined = pd.concat([combined, data], ignore_index=True)
        
        return combined
    
    except Exception as e:
        print(f"Произошла ошибка: {e}")
        return None

table = concat_all_csv_files(path)

Наиболее вероятная кодировка: {'encoding': 'windows-1251', 'language': 'Russian', 'confidence': 1.0}


Объединение данных, с имеющимися справочниками

In [2]:
table_with_OS = pd.read_excel(r'C:\Users\Home\Desktop\Data\Склады\Склады.xlsx', sheet_name='склады')
table_with_OS['Номер_склада'] = table_with_OS['Номер_склада'].astype(str).str.strip()

# Создание словарей для 'Офис' и 'ЛЦ'
office_dict = table_with_OS.set_index('Номер_склада')['ОП'].to_dict()
lc_dict = table_with_OS.set_index('Номер_склада')['ЛЦ'].to_dict()
table['Склад'] = table['Склад'].fillna(0).astype(int).astype(str)

# применяем построчную итерацию для присваения названий, исходя из номеров склада
table['ЛЦ по складу'] = table['Склад'].map(lc_dict).fillna('Неизвестный склад')
table['Офис по складу'] = table['Склад'].map(office_dict).fillna('Неизвестный склад')

#### Обработка даты и времени
Теперь выносим разного рода временные разбивки, чтобы в дальнейшем сделать по ним группировку  
Либо испольщовать в качестве срезов при анализе

In [3]:
table['Дата'] = pd.to_datetime(table['Дата'], format='%d.%m.%Y') 

table['Время'] = pd.to_datetime(
    table['Время'], 
    format='%H:%M', 
    errors='coerce'
).fillna(pd.to_datetime('00:00'))


table['Часы'] = table['Время'].dt.hour
table['Месяц'] = table['Дата'].dt.month

#### Формирование сводных  
Теперь делаем сводные по временным интервалам  
в данном случае - 8 часов это интервал от 8:00 до 8:59

In [4]:
work_hours = table[(table['Часы']>=8) & (table['Часы']<=17)]
pivot_table = work_hours.pivot_table(values='Время',
    index=['ЛЦ по складу','Офис по складу'],
    columns='Часы',
    aggfunc='count',
    fill_value=0)

pivot_table

Часы                           8    9    10   11   12   13   14   15   16  17
ЛЦ по складу Офис по складу                                                  
Воронеж      ВТЦ Волгоград    107  141  149  220  163  166  135  133  139  71
             ВТЦ-2 Воронеж      5  135  123   98   78   86   82   74   64  44
             ОП Белгород      331  607  524  409  284  282  223  172  167  73
             ОП Волгоград      95  186  271  359  310  322  205  147  111  18
             ОП Волжский       94  216  167  170  110  141  210  168  184  79
...                           ...  ...  ...  ...  ...  ...  ...  ...  ...  ..
Челябинск    Златоуст          96   97   96   78  104   88   60   53   12   0
             ОП Магнитогорск  243  171  138  135  108   82   56   62    2   0
             ОП-1 Челябинск   346  422  271  279  206  170  116   44   26   0
             ОП-2 Челябинск   463  338  217  179  195  125  101   47   31   0
             ОП-3 Челябинск   605  671  396  391  307  255   93   73   26   0

[176 rows x 10 columns]

In [5]:
pivot_table_all = table.pivot_table(values='Время',
    index=['ЛЦ по складу','Офис по складу'],
    columns='Часы',
    aggfunc='count',
    fill_value=0).reset_index()

pivot_table_all

Часы,ЛЦ по складу,Офис по складу,0,1,2,3,4,5,6,7,...,13,14,15,16,17,18,19,20,21,22
0,Воронеж,ВТЦ Волгоград,2,0,0,0,0,0,0,2,...,166,135,133,139,71,4,0,0,0,0
1,Воронеж,ВТЦ-2 Воронеж,2,0,0,0,0,0,0,0,...,86,82,74,64,44,36,0,0,0,0
2,Воронеж,ОП Белгород,14,0,0,0,0,0,0,0,...,282,223,172,167,73,0,0,0,0,0
3,Воронеж,ОП Волгоград,38,0,0,0,0,0,0,0,...,322,205,147,111,18,0,0,0,0,0
4,Воронеж,ОП Волжский,0,0,0,0,0,0,0,0,...,141,210,168,184,79,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
171,Челябинск,Златоуст,23,0,0,0,0,0,3,83,...,88,60,53,12,0,0,0,0,0,0
172,Челябинск,ОП Магнитогорск,98,0,0,0,0,0,9,127,...,82,56,62,2,0,0,0,0,0,0
173,Челябинск,ОП-1 Челябинск,50,0,0,0,0,0,11,244,...,170,116,44,26,0,0,0,0,0,0
174,Челябинск,ОП-2 Челябинск,27,0,0,0,0,0,54,547,...,125,101,47,31,0,0,0,0,0,0


### Добавление признака с возмоным фродом

важно в данном случае отслеживать заказы которые могли быть созданы в один период  
в данном случае подозрительно большим кол-вом заказов в час считается ситуация при которой  
среднее значение в точке с 9 до 18 часов превышено на два стандартных отклонения.  
то есть > mean() + std()*2

In [6]:
# Выбираем период рабочего времени точек - с 9 до 18
columns_to_sum = pivot_table_all.columns[12:22]
pivot_table_all['среднее значение в точке с 9 до 18'] = pivot_table_all[columns_to_sum].mean(axis=1).round(2)
pivot_table_all['стандарт отклон на точке с 9 до 18'] = pivot_table_all[columns_to_sum].std(axis=1)
pivot_table_all['всего заказов на точке с 9 до 18'] = pivot_table_all[columns_to_sum].sum(axis=1)

pivot_table_all['верхняя граница допустимого кол-ва заказов'] = pivot_table_all['среднее значение в точке с 9 до 18'] + \
                                                                pivot_table_all['стандарт отклон на точке с 9 до 18']*2
def detect_fraud(row):
    return 1 if any(row[columns_to_sum] > row['верхняя граница допустимого кол-ва заказов']) else 0

pivot_table_all['возможный фрод'] = pivot_table_all.apply(detect_fraud, axis=1)

pivot_table_all

Часы,ЛЦ по складу,Офис по складу,0,1,2,3,4,5,6,7,...,18,19,20,21,22,среднее значение в точке с 9 до 18,стандарт отклон на точке с 9 до 18,всего заказов на точке с 9 до 18,верхняя граница допустимого кол-ва заказов,возможный фрод
0,Воронеж,ВТЦ Волгоград,2,0,0,0,0,0,0,2,...,4,0,0,0,0,118.0,71.319313,1180,260.638627,0
1,Воронеж,ВТЦ-2 Воронеж,2,0,0,0,0,0,0,0,...,36,0,0,0,0,68.5,34.670673,685,137.841346,0
2,Воронеж,ОП Белгород,14,0,0,0,0,0,0,0,...,0,0,0,0,0,213.4,169.696592,2134,552.793184,0
3,Воронеж,ОП Волгоград,38,0,0,0,0,0,0,0,...,0,0,0,0,0,174.3,139.275945,1743,452.851890,0
4,Воронеж,ОП Волжский,0,0,0,0,0,0,0,0,...,0,0,0,0,0,122.9,74.645012,1229,272.190023,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
171,Челябинск,Златоуст,23,0,0,0,0,0,3,83,...,0,0,0,0,0,49.1,42.563547,491,134.227095,0
172,Челябинск,ОП Магнитогорск,98,0,0,0,0,0,9,127,...,0,0,0,0,0,58.3,56.423104,583,171.146208,0
173,Челябинск,ОП-1 Челябинск,50,0,0,0,0,0,11,244,...,0,0,0,0,0,111.2,113.054363,1112,337.308725,0
174,Челябинск,ОП-2 Челябинск,27,0,0,0,0,0,54,547,...,0,0,0,0,0,89.5,85.666342,895,260.832685,0


In [7]:
list_of_offices = pivot_table_all[pivot_table_all['возможный фрод'] == 1]['Офис по складу'].to_list()
print('Список офисов с предполагаемыми нарушениями:')
print(*list_of_offices, sep='\n')

Список офисов с предполагаемыми нарушениями:
ОП Сургут
ОП Нижнекамск
ОП Сыктывкар
ОП Чебоксары
ОП-1 Казань
ОП-1 НН
ВТЦ Чехов
Неизвестный склад
ОП-1 Красноярск
ОП Симферополь
Сочи СВК1
ВТЦ-19 СПб
ВТЦ-3 СПб
Мурманск ОП1


##### Загрузка обработанных данных в файл с отчётом

In [8]:
today = datetime.today().date()

archive_path = f"C:\\Users\\Home\\Desktop\\Data\\Итоговые_отчёты\\контроль времени в офисах продаж{today}.xlsx"
with pd.ExcelWriter(archive_path, engine='openpyxl') as writer:
    pivot_table_all.to_excel(writer, sheet_name='отчёт', index=True)